In this project we will be analyzing the conent of Hackernews posts, and analyze their popularity. 

We will be using the following dataset:
https://www.kaggle.com/hacker-news/hacker-news-posts

It is a filtered dataset that takes a random sample of the submissions to Hackernews that have any activity on the submission (i.e they have a caomment), and then tandomly sampled. 

The headers are as follows:

id: The unique identifier from Hacker News for the post
title: The title of the post
url: The URL that the posts links to, if it the post has a URL
num_points: The number of points the post acquired, calculated as the total number of upvotes minus the total number of downvotes
num_comments: The number of comments that were made on the post
author: The username of the person who submitted the post
created_at: The date and time at which the post was submitted



In [1]:
from csv import reader

opened_file = open('hacker_news.csv')
read_file = reader(opened_file)
hn = list(read_file)

print(hn[:5])


[['id', 'title', 'url', 'num_points', 'num_comments', 'author', 'created_at'], ['12224879', 'Interactive Dynamic Video', 'http://www.interactivedynamicvideo.com/', '386', '52', 'ne0phyte', '8/4/2016 11:52'], ['10975351', 'How to Use Open Source and Shut the Fuck Up at the Same Time', 'http://hueniverse.com/2016/01/26/how-to-use-open-source-and-shut-the-fuck-up-at-the-same-time/', '39', '10', 'josep2', '1/26/2016 19:30'], ['11964716', "Florida DJs May Face Felony for April Fools' Water Joke", 'http://www.thewire.com/entertainment/2013/04/florida-djs-april-fools-water-joke/63798/', '2', '1', 'vezycash', '6/23/2016 22:20'], ['11919867', 'Technology ventures: From Idea to Enterprise', 'https://www.amazon.com/Technology-Ventures-Enterprise-Thomas-Byers/dp/0073523429', '3', '1', 'hswarna', '6/17/2016 0:01']]


We Will now clear up the headers:


In [2]:
headers = hn[0]
hn = hn[1:]

print(headers)

print('\n')
#printing rows to verify

print(hn[:4])


['id', 'title', 'url', 'num_points', 'num_comments', 'author', 'created_at']


[['12224879', 'Interactive Dynamic Video', 'http://www.interactivedynamicvideo.com/', '386', '52', 'ne0phyte', '8/4/2016 11:52'], ['10975351', 'How to Use Open Source and Shut the Fuck Up at the Same Time', 'http://hueniverse.com/2016/01/26/how-to-use-open-source-and-shut-the-fuck-up-at-the-same-time/', '39', '10', 'josep2', '1/26/2016 19:30'], ['11964716', "Florida DJs May Face Felony for April Fools' Water Joke", 'http://www.thewire.com/entertainment/2013/04/florida-djs-april-fools-water-joke/63798/', '2', '1', 'vezycash', '6/23/2016 22:20'], ['11919867', 'Technology ventures: From Idea to Enterprise', 'https://www.amazon.com/Technology-Ventures-Enterprise-Thomas-Byers/dp/0073523429', '3', '1', 'hswarna', '6/17/2016 0:01']]


We will now be looking to filter by the category of posts which have ask HN or show HN , to investigate how well these do on average. These are 2 types of community posts, so we will investiagte if community posts tend to do better than other news articles and submissions. 

Luckily, when it is one of those types of posts, Hackernews starts the tile of the questions with Ask HN, and so we can simply extrapolate the entires which start with that particular string of 'Ask HN'.

To do this we will create a loop to extrapolate a list with the requied titlkes. 


In [9]:
# initiate empty lists to operate on in the loop for the output

ask_posts = []
show_posts = []
other_posts = []

for row in hn:
    title = row[1]
    if title.lower().startswith('ask hn') == True:
        ask_posts.append(row)
    elif title.lower().startswith('show hn') == True:
        show_posts.append(row)
    else:
        other_posts.append(row)
        
print('ask_posts length is ' + str(len(ask_posts)))        
print('show_posts length is ' + str(len(show_posts)))        
print('other_posts length is ' + str(len(other_posts)))

print('proportion of community posts is ' + str((len(ask_posts) + len(show_posts)) / len(hn)))



ask_posts length is 1744
show_posts length is 1162
other_posts length is 17194
proportion of community posts is 0.1445771144278607


Now that we have isolated the types of posts we will see check their engagement on the comment basis:

'num_comments' is column with index 4

Firstly we will investigate the 'ask' posts.


In [21]:
# set a ask comments list, to track total number of asks.
# then find avg comments on an ask post

total_ask_comments = 0

for row in ask_posts:
    num_comments = row[4]
    num_comments = int(num_comments)
    total_ask_comments += num_comments

avg_ask_comments = total_ask_comments / len(ask_posts)

print('avg ask comments: ' + str(avg_ask_comments))

# set a ask comments list, to track total number of asks.
# then find avg comments on an ask post

total_show_comments = 0

for row in show_posts:
    num_comments = row[4]
    num_comments = int(num_comments)
    total_show_comments += num_comments

avg_show_comments = total_show_comments / len(show_posts)

print('avg show comments: ' + str(avg_show_comments))


avg ask comments: 14.038417431192661
avg show comments: 10.31669535283993


From what we can see above, the average Ask comments receive around 14 comments on avareage, vs show comments of 10.3. 

This is somewhat expects, as the ask HN posts, directly ask for community engagement, and such prompting more comments to be created. 



Based on this we wil focus our analysis of the posts of these type, as they seem to acheive good community engagement. 

We will now try to see if the popularity of a post, is related to the time posted. 

To do this we will calculate:
1. Amount of ask posts created in each hour of the dat, alongside the number of comments received
2. Average number of comments the ask posts receive by the hour created

We also note that the the time in the database is stored as a string, so we will want to convert these in to objects, so taht we can manipulate them more easily. 

For Reference, a sample elemt looks like the following:
['12224879', 'Interactive Dynamic Video', 'http://www.interactivedynamicvideo.com/', '386', '52', 'ne0phyte', '8/4/2016 11:52']





In [56]:
from datetime import datetime

result_list = []



for row in ask_posts:
    created_at = row[6]
    comments = row[4]
    result_list.append([created_at, comments])

# Test: check if loop produces desired data     
print(result_list[:5])

#We will now intialize dicitionaries, to track the number of posts in an hour
#We will also count comments in a specific hour. 
# we will then use the two to analyze Comments per posts in a specific hour to gage popularity

counts_by_hour = {}
comments_by_hour = {}


for row in result_list:
    hour = row[0]
    hour_object = datetime.strptime(hour, '%m/%d/%Y %H:%M')
#note : the used implementation of datetime does not require differentaion between padded and unpadded numbers
    target_hour = hour_object.strftime(%H)
    print(target_hour)
    if target_hour not in counts_by_hour:
        counts_by_hour[target_hour] = 1
        comments_by_hour[target_hour] = row[1]
    else:
        counts_by_hour[target_hour] += 1
        comments_by_hour[target_hour] += row[1]




SyntaxError: invalid syntax (<ipython-input-56-5f80652c6fd6>, line 27)